### What have I done
1. Original Random Forest and Decision Tree
```
Random Forest: 500 Trees and 
                    1000 word features-->0.71882
                    2000 word features-->0.72576
                    all features      -->0.73079
Decision Tree: 1000 Features --> 0.61229
```

2. Random Forest and Decision Tree with Parent and Children Structure
```
Random Forest: 3000 features -->  0.73561
               2000 features -->  0.73119
Decision Tree: 3000 features -->  0.61253
               2000 features -->  0.60951
```

3. PCA and DT, RandomForest
```
Decision Tree: 0.58286
           DT: 0.39732
```

##### Parent and Children Structure

In [1]:
import time
notebookstart= time.time()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train_df = pd.read_json('train.json')
test_df = pd.read_json('test.json')
df_index = train_df['id']
y = train_df.cuisine.copy()
train_df.head()

for _, x in train_df.iterrows():
    for ingredient in x.ingredients:
        ingredient = ingredient.replace(',', '')
for _, x in test_df.iterrows():
    for ingredient in x.ingredients:
        ingredient = ingredient.replace(',', '')
        
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features = 3000, tokenizer=lambda x: [i.strip() for i in x.split(',')], lowercase=False)
x = vect.fit_transform(train_df['ingredients'].apply(','.join)) 
x_test = vect.transform(test_df['ingredients'].apply(','.join)) 

feature_name = vect.get_feature_names()
print((x[0,:] + x[1,:]).shape)

def getChildren(feature_name):
    Children=[]
    for i in feature_name:
        if ' ' not in i:
            if i != '' and i != ' ':
                Children.append(i)
    return Children

def UpdateVector1(data, feature_name):
    children=getChildren(feature_name)
    #print(len(children))
    column=feature_name
    deletion=[]
    for i in children:
        #print(children.index(i))
        for j in column:
            if i!=j and i in j:
                data[:,feature_name.index(i)]=data[:,feature_name.index(i)] + data[:,feature_name.index(j)]
                for idx in range(data[:,feature_name.index(i)].shape[1]):
                    if data[idx,feature_name.index(i)]>1:
                        data[idx,feature_name.index(i)]=1
                deletion.append(feature_name.index(j))
    deletion.sort(reverse = True)
    data = data.todense()
    data = np.delete(data,deletion,axis = 1)
    return data

x_ = UpdateVector1(x.copy(),feature_name)
print(x_.shape)
x_test_ = UpdateVector1(x_test.copy(),feature_name)
split = int(0.8 * x.shape[0])  # Use 80% of the data for training and 20% for validation.
print(f'Training, validation split index: {split}')
X_train, y_train, X_valid, y_valid = x_[:split], y[:split], x_[split:], y[split:]

(1, 2000)


/Applications/Anaconda2/anaconda/envs/py36/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


(39774, 550)
Training, validation split index: 31819


In [2]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 500)

In [3]:
from sklearn import tree
model = tree.DecisionTreeClassifier()

In [4]:
model = model.fit(X_train,y_train)

from sklearn.metrics import accuracy_score
train_preds = model.predict(X_train)
valid_preds = model.predict(X_valid)

print(f'Training classification accuracy: {accuracy_score(y_train, train_preds)}')
print(f'Validation classification accuracy: {accuracy_score(y_valid, valid_preds)}')

Training classification accuracy: 0.9956943964298061
Validation classification accuracy: 0.5964802011313639


In [5]:
model.fit(x_, y)
preds = model.predict(x_)
print(f'Training classification accuracy: {accuracy_score(y, preds)}')

test_preds = model.predict(x_test_)

submission = pd.DataFrame()
submission['id'] = test_df.id
submission['cuisine'] = pd.Series(test_preds)
submission.head()
submission.to_csv('random_forest_parent.csv', index=False)

Training classification accuracy: 0.9954744305325087


> Below is 